# Cleanup
This notebook is provided to help you clean up any resources you have created by running through the example. You should also go to the [CloudFormation console](https://console.aws.amazon.com/cloudformation/home) and delete the stack that you created.

**Recommended settings to run this notebook in SageMaker Studio:**

- Image: Data Science
- Kernel: Python3
- Instance type: <font color='blue'>ml.m5.large (2 vCPU + 8 GiB)</font>

**Important Note:**

DO NOT "Run All Cells" on this notebook, manual steps are needed for successful execution

In [ ]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json
import sys
import time

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')
kinesisclient = boto3.client('kinesisanalyticsv2')
kafkaclient = boto3.client('kafka')
cf_client = boto3.client('cloudformation')

In [ ]:
%store -r

### Stop KDA Application

In [ ]:
# Replace "kda-msk-flink-note" if a different name was used when importing Zeppelin notebook in 1_setup.ipynb

KDA_APP_NAME_FILTER = "kda-msk-flink-note"

try:
    response = kinesisclient.list_applications()
except:
    pass

applications = response['ApplicationSummaries']
print(applications)

for application in applications:
    application_name=application['ApplicationName']
    runtime_environment=application['RuntimeEnvironment']
    if "ZEPPELIN" not in runtime_environment and KDA_APP_NAME_FILTER in application_name:
        try:
            stop_response = kinesisclient.stop_application(ApplicationName=application_name, Force=True)
            print(("Applcation stopped:", application_name))
        except:
            pass
        time.sleep(75)

### Delete KDA application

In [ ]:
# Replace "kda-msk-flink-note" if a different name was used when importing Zeppelin notebook in 1_setup.ipynb

app_filter = 'kda-msk-flink-note'

applications = response['ApplicationSummaries']
for application in applications:
    application_name=application['ApplicationName']
    if app_filter in application_name:        
        try:
            describe_response = kinesisclient.describe_application(ApplicationName=application_name)
        except:
            pass
        create_time_stamp = describe_response['ApplicationDetail']['CreateTimestamp']
        try:
            delete_response = kinesisclient.delete_application(ApplicationName=application_name, CreateTimestamp=create_time_stamp)
            print(("Applcation deleted:", application_name))
        except:
            pass
        print(delete_response)
    time.sleep(5)

### Delete feature groups

In [ ]:
try:
    sm.delete_feature_group(FeatureGroupName=aggregate_batch_feature_group_name) 
    print('deleted batch fg')
except:
    pass

try:
    sm.delete_feature_group(FeatureGroupName=aggregate_feature_group_name) # use if needed to re-create
    print('deleted fg')
except:
    pass

In [ ]:
sm.list_feature_groups()

### Remove contents from the S3 bucket

Be sure to set the stack name in cell below to match the name of the CloudFormation stack used to create this SageMaker domain. If you used the default stack name, you should not need to make any updates

![SegmentLocal](images/get-cloudformation-stack-name.gif "get_cf_name")

In [ ]:
# Change stack name if not using default
stackname = 'sagemaker-featurestore-msk-kda-stack'

response = cf_client.describe_stack_resource(
    StackName=stackname,
    LogicalResourceId='KDAStreamingApplicationS3Bucket'
)
kda_s3_bucket = response["StackResourceDetail"]["PhysicalResourceId"]
print(kda_s3_bucket)

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(kda_s3_bucket)
bucket.objects.all().delete()

### Remove the trigger from Lambda

In [ ]:
import boto3

%store -r 

lambda_client = boto3.client('lambda')
paginator = lambda_client.get_paginator('list_event_source_mappings')
mapping_iterator = paginator.paginate(FunctionName=lambda_to_model_arn)

for m in mapping_iterator:
    if len(m['EventSourceMappings']) > 0:
        uuid = m['EventSourceMappings'][0]['UUID']
        print(f'Deleting mapping: {uuid}...')
        lambda_client.delete_event_source_mapping(UUID=uuid)

### Delete the SageMaker Endpoint

In [ ]:
%store -r
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
except:
    pass

### Deleting CloudFormation Template (Manual)

1. After executing above cells, ensure the SageMaker kernel is brought down (by navigating to the circle button on the left menu).
2. Close the SageMaker Studio domain browser window.
3. Delete the CloudFormation template.
4. Due to a known issue, the EFS file share auto-created by SageMaker Studio will cause issues when executing above step. If that happens, the CloudFormation stack delete may fail after executing for a long time. 
5. To successfully delete the stack, manually remove the auto-created EFS volume related to the SageMaker domain by navigating to the console. Also, recommend deleting the auto-created VPC manually so it removes all related ENIs and security groups.

Thanks for working through this content, reach out to us if you have any questions.